In [53]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor

## 目录
- [1. 解释一下GBDT算法的过程](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.2%20GBDT#1-解释一下gbdt算法的过程)
  - [1.1 Boosting思想](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.2%20GBDT#11-boosting思想)
  - [1.2 GBDT原来是这么回事](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.2%20GBDT#12-gbdt原来是这么回事)
- [2. 梯度提升和梯度下降的区别和联系是什么？](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.2%20GBDT#2-梯度提升和梯度下降的区别和联系是什么)
- [3. GBDT的优点和局限性有哪些？](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.2%20GBDT#3-gbdt的优点和局限性有哪些)
  - [3.1 优点](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.2%20GBDT#31-优点)
  - [3.2 局限性](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.2%20GBDT#32-局限性)
- [4. RF(随机森林)与GBDT之间的区别与联系](https://github.com/NLP-LOVE/ML-NLP/tree/master/Machine%20Learning/3.2%20GBDT#4-rf随机森林与gbdt之间的区别与联系)


## 1. 解释一下GBDT算法的过程

GBDT(Gradient Boosting Decision Tree)，全名叫梯度提升决策树，使用的是**Boosting**的思想。

### 1.1 Boosting思想

Boosting方法训练基分类器时采用串行的方式，各个基分类器之间有依赖。它的基本思路是将基分类器层层叠加，每一层在训练的时候，对前一层基分类器分错的样本，给予更高的权重。测试时，根据各层分类器的结果的加权得到最终结果。 

Bagging与Boosting的串行训练方式不同，Bagging方法在训练过程中，各基分类器之间无强依赖，可以进行并行训练。

### 1.2 GBDT原来是这么回事

GBDT的原理很简单，就是所有弱分类器的结果相加等于预测值，然后下一个弱分类器去拟合误差函数对预测值的残差(这个残差就是预测值与真实值之间的误差)。当然了，它里面的弱分类器的表现形式就是各棵树。

举一个非常简单的例子，比如我今年30岁了，但计算机或者模型GBDT并不知道我今年多少岁，那GBDT咋办呢？

- 它会在第一个弱分类器（或第一棵树中）随便用一个年龄比如20岁来拟合，然后发现误差有10岁；
- 接下来在第二棵树中，用6岁去拟合剩下的损失，发现差距还有4岁；
- 接着在第三棵树中用3岁拟合剩下的差距，发现差距只有1岁了；
- 最后在第四课树中用1岁拟合剩下的残差，完美。
- 最终，四棵树的结论加起来，就是真实年龄30岁（实际工程中，gbdt是计算负梯度，用负梯度近似残差）。

**为何gbdt可以用用负梯度近似残差呢？**

回归任务下，GBDT 在每一轮的迭代时对每个样本都会有一个预测值，此时的损失函数为均方差损失函数，

![](https://julyedu-img.oss-cn-beijing.aliyuncs.com/quesbase64155214962034944638.gif)

那此时的负梯度是这样计算的

![](https://julyedu-img.oss-cn-beijing.aliyuncs.com/quesbase64155214962416670973.gif)

所以，当损失函数选用均方损失函数是时，每一次拟合的值就是（真实值 - 当前模型预测的值），即残差。此时的变量是![](https://julyedu-img.oss-cn-beijing.aliyuncs.com/quesbase64155214963633267938.gif)，即“当前预测模型的值”，也就是对它求负梯度。

**训练过程**

简单起见，假定训练集只有4个人：A,B,C,D，他们的年龄分别是14,16,24,26。其中A、B分别是高一和高三学生；C,D分别是应届毕业生和工作两年的员工。如果是用一棵传统的回归决策树来训练，会得到如下图所示结果：

![](https://julyedu-img.oss-cn-beijing.aliyuncs.com/quesbase64153438568191303958.png)

现在我们使用GBDT来做这件事，由于数据太少，我们限定叶子节点做多有两个，即每棵树都只有一个分枝，并且限定只学两棵树。我们会得到如下图所示结果：

![](https://julyedu-img.oss-cn-beijing.aliyuncs.com/quesbase64153438570529256895.png)

在第一棵树分枝和图1一样，由于A,B年龄较为相近，C,D年龄较为相近，他们被分为左右两拨，每拨用平均年龄作为预测值。

- 此时计算残差（残差的意思就是：A的实际值 - A的预测值 = A的残差），所以A的残差就是实际值14 - 预测值15 = 残差值-1。
- 注意，A的预测值是指前面所有树累加的和，这里前面只有一棵树所以直接是15，如果还有树则需要都累加起来作为A的预测值。

然后拿它们的残差-1、1、-1、1代替A B C D的原值，到第二棵树去学习，第二棵树只有两个值1和-1，直接分成两个节点，即A和C分在左边，B和D分在右边，经过计算（比如A，实际值-1 - 预测值-1 = 残差0，比如C，实际值-1 - 预测值-1 = 0），此时所有人的残差都是0。残差值都为0，相当于第二棵树的预测值和它们的实际值相等，则只需把第二棵树的结论累加到第一棵树上就能得到真实年龄了，即每个人都得到了真实的预测值。

换句话说，现在A,B,C,D的预测值都和真实年龄一致了。Perfect！

- A: 14岁高一学生，购物较少，经常问学长问题，预测年龄A = 15 – 1 = 14
- B: 16岁高三学生，购物较少，经常被学弟问问题，预测年龄B = 15 + 1 = 16
- C: 24岁应届毕业生，购物较多，经常问师兄问题，预测年龄C = 25 – 1 = 24
- D: 26岁工作两年员工，购物较多，经常被师弟问问题，预测年龄D = 25 + 1 = 26

所以，GBDT需要将多棵树的得分累加得到最终的预测得分，且每一次迭代，都在现有树的基础上，增加一棵树去拟合前面树的预测结果与真实值之间的残差。

## 2. 梯度提升和梯度下降的区别和联系是什么？ 

下表是梯度提升算法和梯度下降算法的对比情况。可以发现，两者都是在每 一轮迭代中，利用损失函数相对于模型的负梯度方向的信息来对当前模型进行更 新，只不过在梯度下降中，模型是以参数化形式表示，从而模型的更新等价于参 数的更新。而在梯度提升中，模型并不需要进行参数化表示，而是直接定义在函 数空间中，从而大大扩展了可以使用的模型种类。

![](http://wx3.sinaimg.cn/mw690/00630Defgy1g4tdwhqzsdj30rp0afdho.jpg)

## 3. **GBDT**的优点和局限性有哪些？ 

### 3.1 优点

1. 预测阶段的计算速度快，树与树之间可并行化计算。
2. 在分布稠密的数据集上，泛化能力和表达能力都很好，这使得GBDT在Kaggle的众多竞赛中，经常名列榜首。 
3. 采用决策树作为弱分类器使得GBDT模型具有较好的解释性和鲁棒性，能够自动发现特征间的高阶关系。

### 3.2 局限性

1. GBDT在高维稀疏的数据集上，表现不如支持向量机或者神经网络。
2. GBDT在处理文本分类特征问题上，相对其他模型的优势不如它在处理数值特征时明显。 
3. 训练过程需要串行训练，只能在决策树内部采用一些局部并行的手段提高训练速度。 

## 4. RF(随机森林)与GBDT之间的区别与联系

**相同点**：

- 都是由多棵树组成，最终的结果都是由多棵树一起决定。
- RF和GBDT在使用CART树时，可以是分类树或者回归树。

**不同点**：

- 组成随机森林的树可以并行生成，而GBDT是串行生成
- 随机森林的结果是多数表决表决的，而GBDT则是多棵树累加之和
- 随机森林对异常值不敏感，而GBDT对异常值比较敏感
- 随机森林是减少模型的方差，而GBDT是减少模型的偏差
- 随机森林不需要进行特征归一化。而GBDT则需要进行特征归一化



### 获取训练数据

In [54]:
train_feature = np.genfromtxt("train_feat.txt",dtype=np.float32)
num_feature = len(train_feature[0])
train_feature = pd.DataFrame(train_feature)

train_label = train_feature.iloc[:, num_feature - 1]
train_feature = train_feature.iloc[:, 0:num_feature - 2]
train_feature

,0,1,2,3,4,5,6,7
0,0.005988,0.569231,0.647059,0.951220,-0.225434,0.837989,0.357258,-0.003058
1,0.161677,0.743195,0.682353,0.960976,-0.086705,0.780527,0.282945,0.149847
2,0.113772,0.744379,0.541176,0.990244,-0.005780,0.721468,0.434110,-0.318043
3,0.053892,0.608284,0.764706,0.951220,-0.248555,0.821229,0.848604,-0.003058
4,0.173653,0.866272,0.682353,0.951220,0.017341,0.704709,-0.021002,-0.195719


In [55]:
train_label

0    320.0
1    361.0
2    364.0
3    336.0
4    358.0
Name: 9, dtype: float32

### 获取测试数据

In [56]:
test_feature = np.genfromtxt("test_feat.txt",dtype=np.float32)
num_feature = len(test_feature[0])
test_feature = pd.DataFrame(test_feature)

test_label = test_feature.iloc[:, num_feature - 1]
test_feature = test_feature.iloc[:, 0:num_feature - 2]
test_feature

,0,1,2,3,4,5,6,7
0,0.005988,0.569231,0.647059,0.951220,-0.225434,0.837989,0.357258,-0.003058
1,0.161677,0.743195,0.682353,0.960976,-0.086705,0.780527,0.282945,0.149847
2,0.113772,0.744379,0.541176,0.990244,-0.005780,0.721468,0.434110,-0.318043
3,0.053892,0.608284,0.764706,0.951220,-0.248555,0.821229,0.848604,-0.003058
4,0.173653,0.866272,0.682353,0.951220,0.017341,0.704709,-0.021002,-0.195719


In [57]:
test_label

0    320.0
1    361.0
2    364.0
3    336.0
4    358.0
Name: 9, dtype: float32

### GBDT模型建立

In [58]:
gbdt = GradientBoostingRegressor(
  loss = 'ls'
, learning_rate = 0.1
, n_estimators = 100
, subsample = 1
, min_samples_split = 2
, min_samples_leaf = 1
, max_depth = 3
, init = None
, random_state = None
, max_features = None
, alpha = 0.9
, verbose = 0
, max_leaf_nodes = None
, warm_start = False
)

gbdt.fit(train_feature, train_label)
pred = gbdt.predict(test_feature)
total_err = 0

for i in range(pred.shape[0]):
    print('pred:', pred[i], ' label:', test_label[i])
print('均方误差:', np.sqrt(((pred - test_label) ** 2).mean()))

pred: 320.0008173984891  label: 320.0
pred: 360.99965033119537  label: 361.0
pred: 363.99928183902097  label: 364.0
pred: 336.0002344322584  label: 336.0
pred: 358.0000159974151  label: 358.0
均方误差: 0.0005218003748239915
